# Goal
Take the mueller beam simulation from MuellerBeamSimplified and turn it from a 5 dimensional fits file (ra, dec, freq,i, j) where i and j represent the 4 x 4 mueller matrix into a modified fits file into a 3 dimensional fits file (ra, dec, freq, i, j) where the 4 x 4 (i x j) matrix is spilt into 16 different fits files

In [1]:
import numpy as np, healpy as hp, matplotlib.pyplot as plt
import pygsm,time,copy
import pyuvdata.utils as uvutils
import operator,subprocess,argparse
import os,sys,glob,yaml
import json,itertools,shutil

import colorcet as cc
ccc = cc.m_cyclic_grey_15_85_c0_s25
import matplotlib
import cmocean

from astropy.coordinates import EarthLocation
from mpl_toolkits.axes_grid1 import AxesGrid
from astropy.io import fits
from matplotlib import gridspec
from astropy import wcs
from pyuvdata import UVData
from datetime import datetime
from collections import OrderedDict as odict
from astropy.time import Time
from astropy_healpix import HEALPix
from astropy.coordinates import SkyCoord, Galactic
from astropy import units as u
from astropy import constants as c
from copy import deepcopy
from sympy import *
init_printing(use_unicode=True)
from sympy.physics.quantum import TensorProduct


# Works only in python 2 env
sys.path.insert(0,'/home/gonzalo/Desktop/workstation/cst2ijones/')
from cst2ijones.jones_matrix_field  import InstrumentalJonesMatrixField
from cst2ijones.plotting import PlotMueller
from cst2ijones import spherical_coordinates_basis_transformation as scbt
%matplotlib notebook

In [2]:
def StokesMatrix(n):
    if n not in [0,1,2,3]: raise Exception('Input must be an integer in [0,1,2,3]')
    
    if n == 0:
        p = np.array([[1.,0],[0.,1.]])
    elif n == 1:
        p = np.array([[1.,0],[0,-1.]])
    elif n == 2:
        p = np.array([[0,1.],[1.,0]])
    elif n == 3:
        p = np.array([[0., -1j],[1j,0]])
    
    return p

# given some frequency n, this function returns the proper file name.
def txtname(n):
    if n not in range(50,251):
        raise ValueError('no data at that frequency.')
    fname = '/home/gonzalo/Desktop/workstation/NF_Simulations/Radiation patterns/E-field pattern - Rigging height 4.9 m/HERA_4.9m_E-pattern_ {}MHz.txt'.format(str(n))
    
    return fname

# Build Mueller Matrix elements from the pauli spin matrix given some jones matrix, J, at a certain frequency.
def MuellerMatrixElement(J,i,j):
    
    Pi = StokesMatrix(i)
    Pj = StokesMatrix(j)
    
    M_ij = np.einsum('...ab,...bc,...cd,...ad',Pi,J,Pj,J.conj()) / 2. #fancy way of taking the transpose
    
    M_ij = np.real(M_ij)
    
    return M_ij

<b> Lets start by loading in a fits object from a Hera image we are working with <b>

In [3]:
#Note, this image is a 4D array with (stokes, freq, ra,dec)
path = '/home/gonzalo/Desktop/workstation/casa_work/'
fitsobject = path+'zen.2458098.52817.HH.calibrated.uvh5_image/zen.2458098.52817.HH.calibrated.uvh5.image.image.fits'

data, header = fits.getdata(fitsobject, header=True) 

# Lets create the mueller matrix and turn it into a fits file with 5 dimensions

In [4]:
nu0 = 150
nu_nodes = range(nu0-5,nu0+6)
input_files = [txtname(n) for n in nu_nodes]
J = InstrumentalJonesMatrixField(input_files, nu_nodes)


#Now we will load in the simulation of the mueller beam
nu0 = 150
nu_axis = np.array([nu0])
npix_sq = header['NAXIS1']



# Parse the WCS keywords in the primary HDU
w = wcs.WCS(header)

# This won't work for rectangular images
npix_sq = header['NAXIS1']

# Convert Equitorial Coordinates to Spherical Coordinates grid
xpix, ypix = np.meshgrid(np.arange(npix_sq),np.arange(npix_sq), indexing='xy')
#np.meshgrid(np.arange(1,npix+1),np.arange(1,npix+1), indexing='xy')
# Should we start from 0 or 1?  aagh
#Right ascension and declination as seen on the inside of the celestial sphere
ra, dec, dummy_freq, dummy_stokes = w.all_pix2world(xpix, ypix,1,1,1) 
c_icrs = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs') # Coordinates Object

# Convert ra/dec to phi/theta
# don't use ravel because altering the values could change the original data.
theta_grid = np.pi/2. - c_icrs.dec.radian 
theta_flat = np.reshape(theta_grid,-1)

#phi_grid = np.radians(header['CRVAL1']) - c_icrs.ra.radian # azimuthal angle limits [0,2*np.pi]
phi_grid = c_icrs.ra.radian.mean() - c_icrs.ra.radian
phi_flat = np.reshape(phi_grid,-1)

# Calculate rotation around -y (?) to get the beam to point at declination = latitude
z0_cza = np.radians(120.7215)
RotAxis = np.array([0,-1,0])
RotAngle = z0_cza
R_z0 = scbt.rotation_matrix(RotAxis, RotAngle)

theta_hor, phi_hor = scbt.spherical_coordinates_map(R_z0, theta_flat, phi_flat)
phi_hor = 2.*np.pi - phi_hor
ijones_sq = np.reshape(J(nu_axis, theta_hor, phi_hor, R_z0.T),(len(nu_axis),npix_sq, npix_sq, 2, 2))

Set OBSGEO-B to   -30.721389 from OBSGEO-[XYZ].
Set OBSGEO-H to     1496.994 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


Setting interpolant.


In [5]:
# Generate Simulation of the Mueller Matrix
i_index,j_index = 4,4
nchan = len(nu_axis)
new_mueller = np.zeros((npix_sq,npix_sq,nchan,4,4),dtype=np.float64) # [ra pixel,dec pixel, freq,i,j]

for f in range(ijones_sq.shape[0]):
    for i in range(i_index):
        for j in range(j_index):
            new_mueller[:,:,f,i,j] = MuellerMatrixElement(ijones_sq[f],i,j)
hdu_mueller = fits.PrimaryHDU(data=new_mueller.T)

# We just created the 5D mueller fits file, but we have to add the wcs header from our hera image to it

In [7]:
#We will want to transfer wcs information into this new header
wcs_header = w.to_header()
for k in wcs_header.keys():
    hdu_mueller.header[k] = wcs_header[k]

In [8]:
#Lets save the following 16 fits files
iarr = [0,0,0,0,1,1,1,1,2,2,2,2,3,3,3,3]
jarr = [0,1,2,3,0,1,2,3,0,1,2,3,0,1,2,3]
mueller_matrixes = []

filenames = []
for i,j in zip(iarr,jarr):
    mueller_matrixes.append(fits.PrimaryHDU(data=hdu_mueller.data[i][j][0][:][:],header=hdu_mueller.header))
    filenames.append('/home/gonzalo/Desktop/workstation/MuellerMatrixBeam{}{}_{}MHz.fits'.format(i,j,nu0))

for m,files in zip(range(len(mueller_matrixes)),filenames):        
    mueller_matrixes[m].writeto(files,overwrite=True)